In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from utility.types import (
    RebalanceFrequencyEnum,
    AllocationMethodsEnum,
    RegimeDetectionModels,
)

from data.universe import Universe
from data.benchmark import Benchmark

from backtest.backtest import Backtester

In [ ]:
bench_obj = Benchmark(RebalanceFrequencyEnum.MONTH_END)
bench_obj.benchmark_perf

In [ ]:
univ_obj = Universe(keep_only_benchmark_universe=True)
universe = univ_obj.get_universe_returns()
universe

In [ ]:
bk_tester = Backtester(
    universe_returns=univ_obj.get_universe_returns(),
    market_returns=univ_obj.get_universe_returns()["SPTR500N"],
    benchmark_returns=bench_obj.benchmark_returns,
)


ptf_and_bench, ptf_weights_equal_weight, regimes, metrics_df = bk_tester.run_backtest(
    allocation_type=AllocationMethodsEnum.EQUALLY_WEIGHTED,
    rebalance_frequency=RebalanceFrequencyEnum.MONTH_END,
    market_regime_model=RegimeDetectionModels.HIDDEN_MARKOV_MODEL,
    bearish_leverage_by_securities={k: 1 for k in universe.columns},
    bullish_leverage_by_securities={k: 1 for k in universe.columns},
    verbose=False,
)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(25, 10))


ax.plot(regimes.index, regimes.next_beta, label="Beta")
ax_l = ax.twinx()
ax_l.fill_between(
    regimes.index,
    regimes["Regime"],
    alpha=0.2,
    color="red",
    step="pre",
)
ax.set_xlabel("Datetime", fontsize=15)
ax.set_ylabel("Beta", fontsize=15)
ax.set_title(f"Dynamic beta evolution", fontsize=20)
ax.grid()
ax.legend(fontsize=15)